<a href="https://colab.research.google.com/github/AmazingGates/Best-Reads/blob/main/MedicalChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ctransformers

In [ ]:
pip install sentence-transformers

In [ ]:
pip install pinecone-client

In [ ]:
pip install langchain

In [ ]:
pip install langchain-pinecone

In [ ]:
pip install langchain_community

In [ ]:
pip install flask

In [ ]:
pip install pypdf

In [ ]:
import os

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain import PromptTemplate

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
from pinecone import Pinecone as PP, ServerlessSpec

In [ ]:
import pinecone

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.llms import CTransformers

In [ ]:
from ctransformers import AutoModelForCausalLM

In [ ]:
os.environ["PINECONE_API_KEY"] = "2307a9eb-6c38-4d1e-aff0-03e3f519c2d1"

In [ ]:
#api_key = "2307a9eb-6c38-4d1e-aff0-03e3f519c2d1"

In [ ]:
#os.environ["PINECONE_API_ENV"] = "Severless"

In [ ]:
def load_pdf(Data):
    loader = DirectoryLoader(Data,
                    glob = "*.pdf",
                    loader_cls = PyPDFLoader)
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf("sample_data")

In [ ]:
# To double check that our data was extracted successfully we can run our variable with the load function
#stored in it.

# Note: This is only confirmation and isn't a necessity so we can comment this piece of code out once
#we are done with it.

extracted_data

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)

In [ ]:
print("Length of my chunk:", len(text_chunks))

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:
pc = PP(api_key = os.environ["PINECONE_API_KEY"])

index_name = "medical-chatbot"

In [ ]:
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)


In [ ]:
query = "What is Acne?"

In [ ]:
docs = docsearch.similarity_search(query, k = 3)

In [ ]:
print("Result", docs)

In [ ]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
llm = CTransformers(
    model = "TheBloke/Llama-2-7B-Chat-GGML",
    model_file="llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    max_new_tokens = 512,
    temperature = 0.8
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, # Now using the CTransformers wrapper
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs = {"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query": user_input})
    print("Response : ", result["result"])